In [1]:
import numpy as np
import os
import glob
import librosa
import torch
import torchaudio

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

From 
- https://pytorch.org/audio/0.10.0/pipelines.html#wav2vec-2-0-hubert-representation-learning 
- https://pytorch.org/audio/0.10.0/pipelines.html#wav2vec-2-0-hubert-fine-tuned-asr 

In [2]:
# Load the model
bundle = torchaudio.pipelines.HUBERT_ASR_LARGE # or HUBERT_ASR_XLARGE 

model = bundle.get_model()

# encoder_embed_dim = 768 # HUBERT_BASE (see https://pytorch.org/audio/0.10.0/_modules/torchaudio/pipelines/_wav2vec2.html#Wav2Vec2Bundle)
encoder_embed_dim = 1024 # HUBERT_LARGE

#### Register forward hook to get the output of the encoder and acoustic features only

The function extract_features() returns too many layers

In [3]:
def get_features(name):
    def hook(model, input, output):
        if name == "acoustic":
            output, _ = output # second output is optional 
        features[name] = output.detach() # type: ignore (pylance bug)
    return hook

model.feature_extractor.register_forward_hook(get_features("acoustic"))
model.encoder.register_forward_hook(get_features("transformer"))

In [4]:
for audio_fn in glob.glob("/homes/lm004/commercials/annotated_commercials/*.mp3"):
    features = {}

    y, _ = librosa.core.load(audio_fn, sr=bundle.sample_rate, res_type='kaiser_fast')
    y = torch.unsqueeze(torch.from_numpy(y), dim=0)

    boh = model(y)

    break

In [7]:
features['acoustic'].shape # (1, 490, 512)


torch.Size([1, 490, 512])

In [6]:
features['transformer'].shape # (1, 490, 1024)

torch.Size([1, 490, 1024])

In [22]:
features_rate = 0.02 # 20ms, see paper
assert int(y.shape[1]//(features_rate*bundle.sample_rate)) == features['transformer'].shape[1]
assert int(y.shape[1]//(features_rate*bundle.sample_rate)) == features['acoustic'].shape[1]